# Анализ дорожно-транспортных происшествий в России (полные данные)

## Описание проекта
В этом ноутбуке мы повторяем анализ из `02_exploratory_analysis_sample.ipynb`, но на полном наборе данных. Данный проект представляет собой разведочный анализ данных о ДТП в России. Целью является выявление ключевых тенденций, распределения аварий по регионам, времени и условиям, а также оценка тяжести происшествий. Анализ основан на выборочных очищенных данных, предоставленных для воспроизводимости без использования облачных источников.

## Цели анализа
- Изучить динамику числа ДТП и погибших за несколько лет.
- Проанализировать распределение аварий по регионам, времени суток, погодным условиям, участникам и пр.
- Оценить связь факторов (тип ДТП, возраст транспорта, нарушения) и тяжесть происшествий.

## Структура данных
- `accidents`: Основной набор данных о ДТП, включая дату, регион, тип и тяжесть.
- `participants`: Данные об участниках происшествий, включая роль и нарушения.
- `vehicles`: Информация о транспортных средствах, включая возраст и категорию.
- Визуализации сохраняются в папке `../figures/`. Статические — в формате PNG, интерактивные — в формате HTML.
- Данные скачиваются с Яндекс.Диска, сохраняются в SQLite-базу `../data/crash_database.db`
- 
## Источники данных
см. `../README.md`

In [2]:
# Загрузка библиотек и утилит
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys

sys.path.append(os.path.abspath("../scripts")) # Добавляем путь к скриптам

from utils import (
    load_full_data_to_sqlite,
    run_query,
    save_png,
    save_html
)

# Пути для сохранения визуализаций
FIG_DIR = "../figures"
FIG_INT_DIR = "../figures/interactive"

## Загрузка данных
В следующей ячейке мы загружаем файлы из облака в базу данных SQLite для оптимизации работы кода

In [4]:
#%% Загрузка данных в SQLite
accidents_url    = "https://disk.yandex.ru/d/yPdgwafR_2xElg"
participants_url = "https://disk.yandex.ru/d/YeyKLfXuETaEUQ"
vehicles_url     = "https://disk.yandex.ru/d/NJApFGWb85CWVQ"

conn = load_full_data_to_sqlite(
    accidents_url,
    participants_url,
    vehicles_url,
    db_path="../data/crash_database.db"
)

# Динамика числа ДТП по годам и месяцам

## Описание задачи

Следующая ячейка анализирует многолетнюю динамику числа дорожно-транспортных происшествий (ДТП) в России на основе выборочных данных. Цель — выявить тенденции изменения количества аварий и погибших по месяцам и годам.

## Данные

Используются данные из таблицы `accidents`, агрегированные по годам и месяцам с расчётом общего числа ДТП и погибших.

In [7]:
# Динамика числа ДТП по годам и месяцам (интерактивно)
import sys, os
import plotly.graph_objects as go
from utils import run_query, save_html

# SQL-запрос: месячная агрегация
sql_monthly = """
SELECT
  CAST(strftime('%Y', datetime) AS INTEGER) AS year,
  CAST(strftime('%m', datetime) AS INTEGER) AS month,
  COUNT(*)                                 AS accident_count,
  SUM(dead_count)                          AS dead_count
FROM accidents
GROUP BY year, month
ORDER BY year, month
"""
monthly_stats = run_query(conn, sql_monthly)

# готовим подписи для оси X — первый месяц каждого года
time_labels   = []
tick_positions = []
current_year  = None
for i, (y, m) in enumerate(zip(monthly_stats["year"], monthly_stats["month"])):
    if y != current_year:
        time_labels.append(f"{y}-{m:02d}")
        tick_positions.append(i)
        current_year = y

# строим интерактивный график
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(len(monthly_stats))),
    y=monthly_stats["accident_count"],
    mode="lines+markers",
    name="Число ДТП",
    line=dict(color="blue", width=2),
    marker=dict(size=6)
))

# вертикальные линии разделения годов
for pos in tick_positions[1:]:
    fig.add_shape(
        type="line",
        x0=pos, y0=0,
        x1=pos, y1=monthly_stats["accident_count"].max(),
        line=dict(color="gray", width=1, dash="dash")
    )

# оформление
fig.update_layout(
    title="Динамика числа ДТП по годам и месяцам",
    xaxis_title="Год–месяц",
    yaxis_title="Число ДТП",
    xaxis=dict(
        tickmode="array",
        tickvals=tick_positions,
        ticktext=time_labels,
        tickangle=0
    ),
    yaxis=dict(showgrid=True, gridcolor="#e0e0e0"),
    width=800,
    height=400
)

fig.show()

# сохраняем в HTML для интерактивного просмотра
save_html(fig, os.path.join(FIG_INT_DIR, "monthly_accidents.html"))

Query returned 113 rows


'\n# сохраняем в HTML для интерактивного просмотра\nsave_html(fig, os.path.join(FIG_DIR, "monthly_accidents.html"))\n'

## Выводы

- За рассматриваемый период (2015–2024) наблюдается тенденция к постепенному снижению общего числа ДТП.
- Аварийность демонстрирует серьезные сезонные колебания.
- Больше всего аварий приходится на июль-сентябрь, а меньше всего на февраль-май.

# Распределение ДТП по месяцам

В этом блоке мы:
- С помощью SQL извлекаем из базы число ДТП и сумму погибших по годам и месяцам;
- С помощью `calendar.monthrange` определяем число дней в каждом месяце;
- Вычисляем среднее число ДТП в день для каждого месяца с учётом разной длины месяцев;
- Агрегируем результаты по месяцам календарного года, чтобы получить среднее дневное число ДТП в январе, феврале и т.д.;
- Цель — выявить сезонные закономерности, рассчитав среднедневное число ДТП для каждого месяца за все годы.